In [10]:
from pathlib import Path
import os

from jinja2.lexer import float_re

STUDENT = 'mmr497'

OUTLIERS_PATH = Path('./outliers2/')
INTERMEDIATE_PATH = Path('./intermediate_datafiles/')
os.chdir(f'/home/{STUDENT}/')
EXPERIMENT_DIR = 'ML4QS-Vehicle-NT-130625'
from util.VisualizeDataset import VisualizeDataset
from Visualiser import Visualiser as Viz
from outlier_detector import OutlierDetector
from custom_imputer import CustomImputer
from util.util import ignore_actual_time, read_parquet, write_parquet
from DataLoader import PhyboxDatasetLoader
import pandas as pd
import numpy as np

In [2]:
DataViz = VisualizeDataset('example_pipeline.ipynb')
EasyViz = Viz()

In [3]:
# If note done already, load in the Phybox data
# dataset_loader = PhyboxDatasetLoader(STUDENT, exp_dir=EXPERIMENT_DIR, overwrite_loader=False)
# datasets = dataset_loader.create_all_datasets(overwrite=False)
# write_parquet(datasets, INTERMEDIATE_PATH / 'ML4QS_combined_results_example.parquet')

In [5]:
# Load intermediate df
intermediate_df = read_parquet('/local/data/mmr497/intermediate_datafiles/ML4QS_combined_results_example.parquet')
intermediate_df.head()

,id,acc_phone_X,acc_phone_Y,acc_phone_Z,lin_acc_phone_X,lin_acc_phone_Y,lin_acc_phone_Z,gyr_phone_X,gyr_phone_Y,gyr_phone_Z,...,mag_phone_Y,mag_phone_Z,proximity_phone_Distance,labeltram,labeltrain,labelwalking,labelmetro,labelbus,labelcar,labelbike
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-06-04 11:41:49.154,0,0.595657,3.355480,9.099531,0.058131,-0.000233,-0.128019,-0.087578,0.173205,0.062614,...,-45.794652,-28.391914,3.75,1,0,0,0,0,0,0
2025-06-04 11:41:49.404,0,0.142101,3.311633,9.272894,0.017935,-0.017567,0.049362,-0.063947,0.214520,-0.022456,...,-44.642985,-28.854507,NaN,1,0,0,0,0,0,0
2025-06-04 11:41:49.654,0,-0.165194,3.236582,9.294871,0.009399,0.005298,0.037026,-0.025154,0.063762,-0.017961,...,-43.936887,-29.601068,NaN,1,0,0,0,0,0,0
2025-06-04 11:41:49.904,0,-0.393992,3.161564,9.303603,-0.027900,-0.024366,0.029343,-0.002146,0.067268,-0.002748,...,-43.764561,-29.912215,NaN,1,0,0,0,0,0,0
2025-06-04 11:41:50.154,0,-0.447120,3.255967,9.296580,0.031913,0.007953,0.058292,0.084816,0.035108,0.011742,...,-43.858307,-29.666903,NaN,1,0,0,0,0,0,0


In [26]:
train_mask = intermediate_df['labeltrain'] == 1
print(intermediate_df.loc[train_mask, ['location_phone_Direction', 'location_phone_Velocity', 'location_phone_Height']].isna().sum() / len(intermediate_df.loc[train_mask]))

location_phone_Direction    1.000000
location_phone_Velocity     0.999109
location_phone_Height       0.990324
dtype: float64


In [6]:
# Let the dataset ignore actual timepoints
intermediate_df = ignore_actual_time(intermediate_df)
intermediate_df.head()

,id,acc_phone_X,acc_phone_Y,acc_phone_Z,lin_acc_phone_X,lin_acc_phone_Y,lin_acc_phone_Z,gyr_phone_X,gyr_phone_Y,gyr_phone_Z,...,labeltram,labeltrain,labelwalking,labelmetro,labelbus,labelcar,labelbike,original_time,time_diff,shifted_time
timestamp,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:00:00.000,0,0.595657,3.355480,9.099531,0.058131,-0.000233,-0.128019,-0.087578,0.173205,0.062614,...,1,0,0,0,0,0,0,2025-06-04 11:41:49.154,00:00:00,00:00:00
1970-01-01 00:00:00.250,0,0.142101,3.311633,9.272894,0.017935,-0.017567,0.049362,-0.063947,0.214520,-0.022456,...,1,0,0,0,0,0,0,2025-06-04 11:41:49.404,00:00:00.250000,00:00:00.250000
1970-01-01 00:00:00.500,0,-0.165194,3.236582,9.294871,0.009399,0.005298,0.037026,-0.025154,0.063762,-0.017961,...,1,0,0,0,0,0,0,2025-06-04 11:41:49.654,00:00:00.250000,00:00:00.500000
1970-01-01 00:00:00.750,0,-0.393992,3.161564,9.303603,-0.027900,-0.024366,0.029343,-0.002146,0.067268,-0.002748,...,1,0,0,0,0,0,0,2025-06-04 11:41:49.904,00:00:00.250000,00:00:00.750000
1970-01-01 00:00:01.000,0,-0.447120,3.255967,9.296580,0.031913,0.007953,0.058292,0.084816,0.035108,0.011742,...,1,0,0,0,0,0,0,2025-06-04 11:41:50.154,00:00:00.250000,00:00:01


In [5]:
if not intermediate_df.index.is_unique:
    print("Warning: DataFrame contains duplicate indices")


In [7]:
# Set outlier detector
out_detector = OutlierDetector(intermediate_df)
# set outlier detector to mixture model, and automatically set outliers to NaN.
# Note that we can pass a col parameter to this function to only apply the outlier detection to a subset of columns.
intermed_df = out_detector.fit_transform("mixture", outlier_behaviour='nan')

Applying mixture models
Applying mixture models for column:acc_phone_X
Applying mixture models
Applying mixture models for column:acc_phone_Y
Applying mixture models
Applying mixture models for column:acc_phone_Z
Applying mixture models
Applying mixture models for column:lin_acc_phone_X
Applying mixture models
Applying mixture models for column:lin_acc_phone_Y
Applying mixture models
Applying mixture models for column:lin_acc_phone_Z
Applying mixture models
Applying mixture models for column:gyr_phone_X
Applying mixture models
Applying mixture models for column:gyr_phone_Y
Applying mixture models
Applying mixture models for column:gyr_phone_Z
Applying mixture models
Applying mixture models for column:mag_phone_X
Applying mixture models
Applying mixture models for column:mag_phone_Y
Applying mixture models
Applying mixture models for column:mag_phone_Z


In [21]:
type_list = intermediate_df.dtypes
type_list['lin_acc_phone_X']

dtype('float64')

In [37]:
for i, j in type_list.items():
    print(i, j)

id int64
acc_phone_X float64
acc_phone_Y float64
acc_phone_Z float64
lin_acc_phone_X float64
lin_acc_phone_Y float64
lin_acc_phone_Z float64
gyr_phone_X float64
gyr_phone_Y float64
gyr_phone_Z float64
location_phone_Latitude float64
location_phone_Longitude float64
location_phone_Height float64
location_phone_Velocity float64
location_phone_Direction float64
location_phone_Horizontal Accuracy float64
location_phone_Vertical Accuracy float64
mag_phone_X float64
mag_phone_Y float64
mag_phone_Z float64
proximity_phone_Distance float64
labeltram int64
labeltrain int64
labelwalking int64
labelmetro int64
labelbus int64
labelcar int64
labelbike int64
original_time datetime64[ns]
time_diff timedelta64[ns]
shifted_time timedelta64[ns]


In [35]:
intermed_df[type_list.index].astype(type_list.values).dtypes

id                                    object
acc_phone_X                           object
acc_phone_Y                           object
acc_phone_Z                           object
lin_acc_phone_X                       object
lin_acc_phone_Y                       object
lin_acc_phone_Z                       object
gyr_phone_X                           object
gyr_phone_Y                           object
gyr_phone_Z                           object
location_phone_Latitude               object
location_phone_Longitude              object
location_phone_Height                 object
location_phone_Velocity               object
location_phone_Direction              object
location_phone_Horizontal Accuracy    object
location_phone_Vertical Accuracy      object
mag_phone_X                           object
mag_phone_Y                           object
mag_phone_Z                           object
proximity_phone_Distance              object
labeltram                             object
labeltrain

In [7]:
test_test = out_detector.fitted_data
test_test.head(20)

,acc_phone_X,acc_phone_Y,acc_phone_Z,lin_acc_phone_X,lin_acc_phone_Y,lin_acc_phone_Z,gyr_phone_X,gyr_phone_Y,gyr_phone_Z,location_phone_Latitude,...,acc_phone_Z_mixture,lin_acc_phone_X_mixture,lin_acc_phone_Y_mixture,lin_acc_phone_Z_mixture,gyr_phone_X_mixture,gyr_phone_Y_mixture,gyr_phone_Z_mixture,mag_phone_X_mixture,mag_phone_Y_mixture,mag_phone_Z_mixture
timestamp,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:00:00.000,0.595657,3.355480,9.099531,0.058131,-0.000233,-0.128019,-0.087578,0.173205,0.062614,52.371481,...,4.304526e-05,2.555560,2.824265,1.956709,0.413878,0.028488,0.172459,3.64619e-05,1.1778e-06,2.22007e-06
1970-01-01 00:00:00.250,0.142101,3.311633,9.272894,0.017935,-0.017567,0.049362,-0.063947,0.214520,-0.022456,NaN,...,2.832031e-05,2.883277,2.932781,1.397305,0.589320,0.026210,5.273844,3.66601e-05,1.35331e-06,2.07286e-06
1970-01-01 00:00:00.500,-0.165194,3.236582,9.294871,0.009399,0.005298,0.037026,-0.025154,0.063762,-0.017961,NaN,...,2.671655e-05,2.913808,2.775227,1.512792,1.040779,0.049039,30.648881,3.82693e-05,1.47204e-06,1.85408e-06
1970-01-01 00:00:00.750,-0.393992,3.161564,9.303603,-0.027900,-0.024366,0.029343,-0.002146,0.067268,-0.002748,NaN,...,2.609635e-05,2.867121,2.955345,1.582919,1680.470676,0.041089,944.383274,3.9012e-05,1.50238e-06,1.76938e-06
1970-01-01 00:00:01.000,-0.447120,3.255967,9.296580,0.031913,0.007953,0.058292,0.084816,0.035108,0.011742,NaN,...,2.659440e-05,2.801758,2.749406,1.312544,0.587433,7.547019,216.381860,3.94172e-05,1.48581e-06,1.83585e-06
1970-01-01 00:00:01.250,-0.578040,3.358048,9.176276,-0.039910,-0.062625,-0.003416,0.008628,-0.006916,-0.043919,NaN,...,3.608692e-05,2.791530,2.862963,1.852255,388.396617,294.990675,0.176500,4.00648e-05,1.72164e-06,2.06882e-06
1970-01-01 00:00:01.500,-0.591695,3.381044,9.227174,-0.054898,0.004569,0.031482,0.000356,-0.005497,-0.018901,NaN,...,3.185163e-05,2.660720,2.782060,1.563571,1811.195171,311.243820,21.503885,4.12932e-05,2.56589e-06,1.97582e-06
1970-01-01 00:00:01.750,-0.558063,3.387859,9.261149,-0.026601,0.002001,0.063562,-0.006353,0.000153,0.001555,52.371520,...,2.920242e-05,2.873617,2.805250,1.262448,838.615479,339.907944,996.213693,4.23736e-05,4.52193e-06,1.86304e-06
1970-01-01 00:00:02.000,-0.534401,3.367672,9.164018,0.004183,-0.006206,-0.004835,0.001990,0.016415,0.000015,NaN,...,3.715331e-05,2.924984,2.869633,1.862429,1657.704093,143.154245,1025.526692,4.32505e-05,6.67784e-06,1.72796e-06


In [8]:
intermed_df.head(20)

,id,acc_phone_X,acc_phone_Y,acc_phone_Z,lin_acc_phone_X,lin_acc_phone_Y,lin_acc_phone_Z,gyr_phone_X,gyr_phone_Y,gyr_phone_Z,...,labeltram,labeltrain,labelwalking,labelmetro,labelbus,labelcar,labelbike,original_time,time_diff,shifted_time
timestamp,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:00:00.000,0,0.595657,3.355480,9.099531,0.058131,-0.000233,-0.128019,-0.087578,0.173205,0.062614,...,1,0,0,0,0,0,0,2025-06-04 11:41:49.154,00:00:00,00:00:00
1970-01-01 00:00:00.250,0,0.142101,3.311633,9.272894,0.017935,-0.017567,0.049362,-0.063947,0.214520,-0.022456,...,1,0,0,0,0,0,0,2025-06-04 11:41:49.404,00:00:00.250000,00:00:00.250000
1970-01-01 00:00:00.500,0,-0.165194,3.236582,9.294871,0.009399,0.005298,0.037026,-0.025154,0.063762,-0.017961,...,1,0,0,0,0,0,0,2025-06-04 11:41:49.654,00:00:00.250000,00:00:00.500000
1970-01-01 00:00:00.750,0,-0.393992,3.161564,9.303603,-0.027900,-0.024366,0.029343,-0.002146,0.067268,-0.002748,...,1,0,0,0,0,0,0,2025-06-04 11:41:49.904,00:00:00.250000,00:00:00.750000
1970-01-01 00:00:01.000,0,-0.447120,3.255967,9.296580,0.031913,0.007953,0.058292,0.084816,0.035108,0.011742,...,1,0,0,0,0,0,0,2025-06-04 11:41:50.154,00:00:00.250000,00:00:01
1970-01-01 00:00:01.250,0,-0.578040,3.358048,9.176276,-0.039910,-0.062625,-0.003416,0.008628,-0.006916,-0.043919,...,1,0,0,0,0,0,0,2025-06-04 11:41:50.404,00:00:00.250000,00:00:01.250000
1970-01-01 00:00:01.500,0,-0.591695,3.381044,9.227174,-0.054898,0.004569,0.031482,0.000356,-0.005497,-0.018901,...,1,0,0,0,0,0,0,2025-06-04 11:41:50.654,00:00:00.250000,00:00:01.500000
1970-01-01 00:00:01.750,0,-0.558063,3.387859,9.261149,-0.026601,0.002001,0.063562,-0.006353,0.000153,0.001555,...,1,0,0,0,0,0,0,2025-06-04 11:41:50.904,00:00:00.250000,00:00:01.750000
1970-01-01 00:00:02.000,0,-0.534401,3.367672,9.164018,0.004183,-0.006206,-0.004835,0.001990,0.016415,0.000015,...,1,0,0,0,0,0,0,2025-06-04 11:41:51.154,00:00:00.250000,00:00:02


In [9]:
# Plot the results
# If you only want to plot specific columns, you can pass a list of columns to the plot_outliers function.
# Because we automatically drop the outlier columns, we use the fitted data from the outlier detector.
outlier_df = out_detector.fitted_data
EasyViz.plot_outliers(outlier_df, outlier_type='mixture', path='outliers_example')

Figure saved to figures/outliers_example/figure_1.png
Figure saved to figures/outliers_example/figure_1.pdf
Figure saved to figures/outliers_example/figure_2.png
Figure saved to figures/outliers_example/figure_2.pdf
Figure saved to figures/outliers_example/figure_3.png
Figure saved to figures/outliers_example/figure_3.pdf
Figure saved to figures/outliers_example/figure_4.png
Figure saved to figures/outliers_example/figure_4.pdf
Figure saved to figures/outliers_example/figure_5.png
Figure saved to figures/outliers_example/figure_5.pdf
Figure saved to figures/outliers_example/figure_6.png
Figure saved to figures/outliers_example/figure_6.pdf
Figure saved to figures/outliers_example/figure_7.png
Figure saved to figures/outliers_example/figure_7.pdf
Figure saved to figures/outliers_example/figure_8.png
Figure saved to figures/outliers_example/figure_8.pdf
Figure saved to figures/outliers_example/figure_9.png
Figure saved to figures/outliers_example/figure_9.pdf
Figure saved to figures/outl

In [12]:
# Create copy of df to do imputation
imputation_df = intermed_df.copy()

# Remove sensor cols we deem bad
bad_sensors = ['proximity']
drop_cols = []
for sensor in bad_sensors:
    drop_cols.extend(imputation_df.columns[imputation_df.columns.str.contains(sensor)])
imputation_df.drop(drop_cols, axis=1, inplace=True)

In [8]:
imputation_df = intermed_df.copy()

In [13]:
# Create imputer object and set imputer method
imputer = CustomImputer('interpolation')
# Fit and transform the imputation_df
imputed_df = imputer.fit_transform(imputation_df, verbose=True)

NaN values before imputation:
acc_phone_X                               0
acc_phone_Y                               0
acc_phone_Z                               0
lin_acc_phone_X                           0
lin_acc_phone_Y                           0
lin_acc_phone_Z                           0
gyr_phone_X                               0
gyr_phone_Y                               0
gyr_phone_Z                               0
location_phone_Latitude               45012
location_phone_Longitude              45012
location_phone_Height                 45090
location_phone_Velocity               45345
location_phone_Direction              45555
location_phone_Horizontal Accuracy    45012
location_phone_Vertical Accuracy      45012
mag_phone_X                            1631
mag_phone_Y                            1631
mag_phone_Z                            1631
original_time                             0
time_diff                                 0
shifted_time                              0
dt

In [18]:
mask = imputed_df['id'] == 5
intermed_df.loc[mask].head()

,id,acc_phone_X,acc_phone_Y,acc_phone_Z,lin_acc_phone_X,lin_acc_phone_Y,lin_acc_phone_Z,gyr_phone_X,gyr_phone_Y,gyr_phone_Z,...,labeltram,labeltrain,labelwalking,labelmetro,labelbus,labelcar,labelbike,original_time,time_diff,shifted_time
timestamp,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:39:10.250,5,3.094904,5.740396,6.527839,0.37013,-0.794324,-0.341846,-0.0146596,0.148807,0.272282,...,0,1,0,0,0,0,0,2025-06-11 09:36:05.865,00:00:00.250000,00:39:10.250000
1970-01-01 00:39:10.500,5,2.788004,4.982781,7.741858,-0.272509,-1.25649,0.909531,-0.242169,-0.0320798,-0.0131683,...,0,1,0,0,0,0,0,2025-06-11 09:36:06.115,00:00:00.250000,00:39:10.500000
1970-01-01 00:39:10.750,5,7.266773,3.497077,5.248535,3.1084,-1.9312,-1.78051,-0.65524,-1.23887,0.22,...,0,1,0,0,0,0,0,2025-06-11 09:36:06.365,00:00:00.250000,00:39:10.750000
1970-01-01 00:39:11.000,5,6.835925,2.437425,2.333575,1.13757,-2.05163,-3.88726,-0.434271,-1.38723,0.0195479,...,0,1,0,0,0,0,0,2025-06-11 09:36:06.615,00:00:00.250000,00:39:11
1970-01-01 00:39:11.250,5,10.402847,1.792154,-1.265389,2.48275,-1.58709,-5.53507,0.343443,-1.91396,0.569948,...,0,1,0,0,0,0,0,2025-06-11 09:36:06.865,00:00:00.250000,00:39:11.250000


In [14]:
imputed_df.isna().sum()

id                                        0
acc_phone_X                               0
acc_phone_Y                               0
acc_phone_Z                               0
lin_acc_phone_X                           0
lin_acc_phone_Y                           0
lin_acc_phone_Z                           0
gyr_phone_X                               0
gyr_phone_Y                               0
gyr_phone_Z                               0
location_phone_Latitude                   0
location_phone_Longitude                  0
location_phone_Height                  5754
location_phone_Velocity               14431
location_phone_Direction              15709
location_phone_Horizontal Accuracy        0
location_phone_Vertical Accuracy          0
mag_phone_X                               0
mag_phone_Y                               0
mag_phone_Z                               0
labeltram                                 0
labeltrain                                0
labelwalking                    

In [10]:
# Write imputated_df to parquet
write_parquet(imputed_df, INTERMEDIATE_PATH / 'ML4QS_imputed_results.parquet')

Successfully written to parquet file at  intermediate_datafiles/ML4QS_imputed_results.parquet 



In [13]:
# Plot the results of the imputation
# We provide both the original dataset and the imputated dataset to show the difference.
# Again, we can specify which cols we want to use.

EasyViz.plot_imputation(imputed_df, imputation_df, imputation_type='interpolation', path='imputation_example')

TypeError: 'in <string>' requires string as left operand, not dict_values

In [14]:
# Plot the resulting dataframe

EasyViz.plot_dataset(imputed_df, plot_type='full', path='plot_dataset_example')

Figure saved to figures/plot_dataset_example/figure_1.png
Figure saved to figures/plot_dataset_example/figure_1.pdf


In [ ]:
# You can use plot_dataset for multiple different types of plotting.

imputed_df_orig_time = imputed_df.copy()
imputed_df_orig_time.set_index('original_time', inplace=True)

# Plot per day (requires the original timestamps)
EasyViz.plot_dataset(imputed_df, plot_type='day', path='plot_dataset_example_day')

# Plot per individual instance
# e.g. Only instances with the label train
example_instances = imputed_df[imputed_df['label'] == 'train'].id.unique()
EasyViz.plot_dataset(imputed_df, instances=example_instances,  plot_type='instance', path='plot_dataset_example_instance')

# Plot a specific slice
# Works best with original timestamps.
# TODO: Finish
# EasyViz.plot_dataset(imputed_df, time_slice=(), plot_type='slice', path='plot_dataset_example_slice')